In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [2]:
def recortar(imagen):
    for im in glob.glob("Usuario/*.jpg"):
        try:
            os.remove(im)
        except OSError as e:
            print(f"Error:{ e.strerror}")    
    image = cv2.imread(imagen,cv2.IMREAD_GRAYSCALE)
    gradX = cv2.Sobel(image, cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(image, cv2.CV_32F, dx=0, dy=1, ksize=-1)
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)
    blurred = cv2.blur(gradient, (9, 9))
    _, thresh = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations=4)
    closed = cv2.dilate(closed, None, iterations=4)
    a=100
    c=0
    Imagen=closed
    while a>2:
        (cntsf, xf) = cv2.findContours(Imagen.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cf = sorted(cntsf, key=cv2.contourArea, reverse=True)[0]
        rectf = cv2.minAreaRect(cf)
        boxf = np.int0(cv2.boxPoints(rectf))
        Xsf = [i[0] for i in boxf]
        Ysf = [i[1] for i in boxf]
        x1f = min(Xsf)
        x2f = max(Xsf)
        y1f = min(Ysf)
        y2f = max(Ysf)
        cropImgf= image[y1f:y2f, x1f:x2f]
        #plt.imshow(cropImgf)
        C=str(c)
        cv2.imwrite("Usuario/imagen"+C+".jpg", cropImgf)
        #HASTA AQUI SE HIZO TODO UNA VEZ
        b=0
        dimensionesf = (closed.shape[0],closed.shape[1])
        arrayf=np.zeros(dimensionesf)
        for i in range(closed.shape[1]):
            for j in range(closed.shape[0]):
                if (x1f < i < x2f and y1f < j < y2f):
                    arrayf[j,i]=0
                else:
                    arrayf[j,i]=Imagen[j,i]
        for i in range(closed.shape[1]):
            for j in range(closed.shape[0]):
                if (arrayf[j,i]>0):
                    b=b+1
        a=(b*100)/(closed.shape[0]*closed.shape[1])
        c=c+1
        Imagen= np.uint8(arrayf)

In [3]:
recortar("Pulpo.png")